# Naive Bayes Sentiment Classification Program

In [2]:
# unit test classifier 
from util import NB_DataHandler
import sys
import time
import matplotlib.pyplot as plt

amazon = "amazon_cells_labelled.txt"
imdb = "imdb_labelled.txt"
yelp = "yelp_labelled.txt"
test_file = input("Input Unit test file name: ")

v2 = NB_DataHandler(amazon, imdb, yelp, quiet=False)
res = v2.test()
print("Initial Train Accuracy:\t{}%".format(str(round(res,3))))

Input Unit test file name: 
amazon_cells_labelled.txt Finished on line 1000 with 10202 words

imdb_labelled.txt Finished on line 1000 with 14300 words

yelp_labelled.txt Finished on line 1000 with 10848 words

=====Summary=====
Unique words:	 -> 3417
Total words:	 -> 28231
Total data:	 -> 3000
Train data:	 -> 2400
Test data:	 -> 600

31 words removed from vocabulary
Log probabilities calculated
Initial Train Accuracy:	81.167%


In [ ]:
# test accuracy over several runs
print("Selecting best model for unit testing...\n")
runs = 100
run = None
max_res = 0
model = None
vals = []
start = time.time()
for i in range(1, runs+1):
    if i%10 == 0:
        t = round(time.time() - start,3)
        print("run {}: {}s".format(i, t))
    v2 = NB_DataHandler(amazon, imdb, yelp)
    res = v2.test()
    if res >= max_res:
        max_res = res
        model = v2
        run = i
    vals.append(res)
    

avg = sum(vals)/len(vals)
finish = time.time()-start
print("Max accuracy on run {}:\t{} average accuracy: {}% \ntime: {}s"
      .format(run, round(max(vals),3), round(avg, 3), round(finish,3)))

r = input("Do you wish to display Accuracy plot?\n[Y/N]: ")
if r.lower() == "y":
    plt.plot(vals)
    plt.show()

In [ ]:
print("beginning unit test on file: {}\n====================================================\n.......\n"format(test_file))

test_res = model.unit_test(test_file)

print("Unit test Accuracy:\t{}%\n".format(round(test_res,3)))

print("Do you wish to see mssclasification report?\nThis will show how the model behaved on missclasified sentences")
i = input("[Y/N]: ")

if i.lower() == "y":
    model.report()

print("End of program")
